In [ ]:
import datetime as dt 
import random
import pandas as pd
import numpy as np
import statistics
from sklearn.metrics import accuracy_score  
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split
from statistics import mean 
from sklearn.ensemble import RandomForestRegressor
import copy
from sklearn.ensemble import RandomForestClassifier
from scipy import stats
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import xgboost as xgb	
from sklearn.utils import resample
seed = 1234
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
from collections import Counter
from itertools import dropwhile
from sklearn.feature_extraction.text import TfidfVectorizer



def data_maker(data):
    
    
    data["created_at1"] = pd.to_datetime(data['created_at'])
    data["created_at"] = data["created_at1"].dt.strftime('%m/%d/%Y')
    days =  (data.created_at.unique())
    datan = pd.DataFrame()
        
    datan["dates"] = days
    #datan.to_csv('C:/Users/simeon/Documents/data_test.csv')    
    
    data = data.set_index("created_at")
    #data.to_csv('C:/Users/simeon/Documents/data_test.csv')   
    tot_tweet =[]
    tot_bull = []
    tot_bear =[]
    grouped_daily_tweets = []
    for i in range(len(days)):
            
        d_use = datan["dates"][i]
        #print(d_use)
        value=str(d_use)
        sub = ((data.loc[[value]]))
        sub["body"]
                
        L = len(sub)
        tot_tweet.append(L)
        sub1 = sub[(sub.sentiment == {'class': 'bullish', 'name': 'Bullish'}) | (sub.sentiment ==  'Bullish')]
        BL = len(sub1)
        sub2 = sub[(sub.sentiment == {'class': 'bearish', 'name': 'Bearish'}) | (sub.sentiment == 'Bearish')]
        BL2 = len(sub2)
        tot_bull.append(BL)
        tot_bear.append(BL2)
        grouped = sub.groupby(["created_at"])["body"].apply(list)
        grouped_daily_tweets.append(grouped[0])
            
    datan["tot_bull"] = tot_bull
    datan["tot_bear"] = tot_bear
    datan["tot_tweet"] = tot_tweet
    datan["tweets"] = grouped_daily_tweets
    datan["mistake"] = 0 
    datan.loc[datan["tot_tweet"] < datan["tot_bear"], "mistake"] = 1
    datan.loc[datan["tot_tweet"] < datan["tot_bull"], "mistake"] = 1
    datan.loc[datan["mistake"] ==1 , "tot_tweet"] = 1 
    datan.loc[datan["mistake"] ==1 , "tot_bull"] = 0 
    datan.loc[datan["mistake"] ==1 , "tot_bear"] = 0 
    datan["polarity"] = ((datan["tot_bull"] - datan["tot_bear"])/datan["tot_tweet"])
    datan["roll_avg"] = datan.polarity.rolling(window=3).mean()
    change = [100 * (b - a) / a for a, b in zip(datan["tot_tweet"][::1], datan["tot_tweet"][1::1])]
    change.append(0)
    datan["percent_change"] = change[:]
    av =  datan.tot_tweet.rolling(window=10).mean()
    datan["mv10"] = (datan["tot_tweet"]/av)
    datan = datan.drop(["mistake"], axis = 1)
    global new_data
    new_data = datan[:]
    
    
    
    
    
    
def stop_remover(Input_data):
    
    Input_data = pd.DataFrame(Input_data)
    Input_data['tweets'] = Input_data['tweets'].astype(str).str.replace('\d+', '')
    
    Input_data['tweets'] = Input_data['tweets'].astype(str).apply(lambda x: " ".join(x.lower() for x in x.split()))
    Input_data['tweets'] = Input_data['tweets'].astype(str).str.replace('[^\w\s]','')
    
    pat = r'\b(?:{})\b'.format('|'.join(stop))
    Input_data['tweets_no_stop'] = Input_data['tweets'].astype(str).str.replace(pat, '')
    Input_data['tweets_no_stop'] = Input_data['tweets_no_stop'].str.replace(r'\s+', ' ')
    
    global new
    new = Input_data[:]
    
    
names = ["PowerShares QQQ Trust, Series 1","SPDR S&P 500 ETF Trust","facebook","Facebook","american express","American Express Company","American Express","Boeing","Boeing Co","Chevron Corporation","Chevron","coca cola","Coca-Cola","Coca Cola","DuPont","Exxon Mobile ","Exxon Mobile Corporation","ExxonMobil","Exxon","General Electric",'general electric',"General Electri Company","Goldman Sachs Group","Goldman Sachs","Goldman Sachs Group inc","goldman","Godlman","home depot","Home Depot","IBM","Intel","Intel Corporation","johnson & johnson","Johnson & Johnson","Johnson","Johnson and Johnson","JP Morgan","JPMorgan","JPMorgan Chase","3M Co","3M","3m","Merck","Merck & Co.", "Microsoft Corporation","microsoft","Microsoft" ,"pfizer","Pfizer" , "Proctor and Gamble", "proctor & gamble","Procter & Gabmle" , "caterpillar", "Caterpillar", "CAT", "cisco","CISCO","Cisco","Cisco Systems","travelers companies","Travelers Companies","UnitedHealth Group","United Technologies Corporation","VISA","visa","Visa","Verizon","verizon","Verizon Communications","Walmrat","walmart","google", "Google", "Nike", "nike", "McDonald's","Mcdonald's", "mcdonald's", "Disney", "disney", "Walt Disney" ]


def name_remover(Input_data):
    
    Input_data = pd.DataFrame(Input_data)
    
    
    
    
    pat = r'\b(?:{})\b'.format('|'.join(names))
    Input_data['bodyn'] = Input_data['body'].astype(str).str.replace(pat, '')
    Input_data['body'] = Input_data['bodyn'].str.replace(r'\s+', ' ')
    global new
    new = Input_data[:]
    
    
tickers = ['$MCD','$DIS','$WMT','$VZ','$V','$UTX','$UNH','$TRV','$CSCO','$CAT','$PG','$PFE','$MSFT','$MRK','$MMM','$JPM','$JNJ','$INTC','$IBM','$HD','$GS','$GE',"$XOM",'$DD','$KO',"$CVX","$BA","$AXP","$FB","$QQQ","$SPY","$NKE"]

def dup_ender(data, ticker = ''):
    tot_tic = []
    stock_tic = []
    
    
    for i in range(len(data)):
        tot = sum(data["body"][i].count(x) for x in (tickers))
        
        stock = data["body"][i].count(ticker) 
        tot_tic.append(tot)
        stock_tic.append(stock)
    data["tot_tic"] = tot_tic
    data["stock_tic"] = stock
    #print( data["tot_tic"])
    #print(data["stock_tic"])
    data["r"] = np.where(data['tot_tic']>data["stock_tic"], 1, 0)
    data["body"] = np.where(data['r'] == 1,'',data["body"])
    #for i in range(len(data)):
        #print(i)
        
       # if data["stock_tic"][i] < data["tot_tic"][i]:
        #    data["body"][i] = ['']
    global new1
    new1 = data[:]
    
    
    
    
    
    
    
def shebang(data, data_P, ticker1 = "",  ):
    name_remover(data)
    data = new[:]
    dup_ender(data, ticker = ticker1)
    data = new1[:]
    data_maker(data)
    datan = new_data[:]
    data_P["1_day_return"] = data_P.CLOSE.pct_change(1)
    data_P['1_day_return'] = data_P['1_day_return'].shift(-1)
    data_P["3_day_return"] = data_P.CLOSE.pct_change(3)
    data_P['3_day_return'] = data_P['3_day_return'].shift(-3)
    data_P["5_day_return"] = data_P.CLOSE.pct_change(5)
    data_P['5_day_return'] = data_P['5_day_return'].shift(-5)
    stop_remover(datan)
    datan = new[:]
    datan['dates'] = pd.to_datetime(datan['dates'], errors='coerce')
    data_P['Date'] = pd.to_datetime(data_P['Date'], errors='coerce')
    datan = datan.set_index('dates')
    data_P = data_P.set_index('Date')
    datan = datan.iloc[::-1]
    new_d = pd.concat([data_P, datan],ignore_index=False, axis = 1 , join = "outer", join_axes = [data_P.index])
    global F
    F = new_d[:]
    
    
    
    
 
#GS = pd.read_json("C:/Users/simeon/Documents/GS.txt")
#GS_P = pd.read_csv("C:/Users/simeon/Documents/GS.csv")
   
#shebang(GS, GS_P, "$GS")  
#GS = F[:]

#GS.to_csv(r'C:/Users/simeon/Documents/GS_test.csv')

#print(GS.dtypes)



#GS['1_day_return'].shift(-1)




GS1 = pd.read_json("C:/Users/simeon/Documents/GS.txt")
#GS1.to_csv(r'C:/Users/simeon/Documents/clean/GS1.csv')



GS2 = pd.read_csv("C:/Users/simeon/Documents/clean/GS_latest.csv")
# foldman sachs updated testing

GS1['created_at']
GS2["date"].astype(str)

print(GS2.dtypes)
print(GS1.dtypes)


#pip install python-dateutil
import datetime
from dateutil import parser

GS2['date'] = GS2["date"].apply(lambda x:  parser.parse(x))


GS = GS1[['created_at','sentiment','body']].copy()
dates = GS2['date'].tolist() + GS1['created_at'].tolist()


# Creating an empty Dataframe with column names only
dfObj = pd.DataFrame(columns=['User_ID', 'UserName', 'Action'])
print("Empty Dataframe ", dfObj, sep='\n')

# Creating an empty Dataframe with column names only
GS0 = pd.DataFrame(columns=['created_at', 'sentiment', 'body'])

GS0['created_at'] = dates

sentiment = GS2['sentiment'].tolist() + GS1['sentiment'].tolist()

GS0['sentiment'] = sentiment

body = GS2['body'].tolist() + GS1['body'].tolist()

GS0['body'] = body

GS = GS0[:]
GS.dtypes


data = GS[:]



GS_P = pd.read_csv("C:/Users/simeon/Documents/GS_P.csv")


GS_P = GS_P.rename(columns={'Close': 'CLOSE'})


NKE = pd.read_json("C:/Users/simeon/Documents/NKE.txt")
NKE_P = pd.read_csv("C:/Users/simeon/Documents/NKE.csv")
MCD = pd.read_json("C:/Users/simeon/Documents/MCD.txt")
MCD_P = pd.read_csv("C:/Users/simeon/Documents/MCD.csv")
DIS = pd.read_json("C:/Users/simeon/Documents/DIS.txt")
DIS_P = pd.read_csv("C:/Users/simeon/Documents/DIS.csv")
WMT = pd.read_json("C:/Users/simeon/Documents/WMT.txt")
WMT_P = pd.read_csv("C:/Users/simeon/Documents/WMT.csv")
VZ = pd.read_json("C:/Users/simeon/Documents/VZ.txt")
VZ_P = pd.read_csv("C:/Users/simeon/Documents/VZ.csv")
V = pd.read_json("C:/Users/simeon/Documents/V.txt")
V_P = pd.read_csv("C:/Users/simeon/Documents/V.csv")
UTX = pd.read_json("C:/Users/simeon/Documents/UTX.txt")
UTX_P = pd.read_csv("C:/Users/simeon/Documents/UTX.csv")
UNH = pd.read_json("C:/Users/simeon/Documents/UNH.txt")
UNH_P = pd.read_csv("C:/Users/simeon/Documents/UNH.csv")
TRV = pd.read_json("C:/Users/simeon/Documents/TRV.txt")
TRV_P = pd.read_csv("C:/Users/simeon/Documents/TRV.csv")
CSCO = pd.read_json("C:/Users/simeon/Documents/CSCO.txt")
CSCO_P = pd.read_csv("C:/Users/simeon/Documents/CSCO.csv")
CAT = pd.read_json("C:/Users/simeon/Documents/CAT.txt")
CAT_P = pd.read_csv("C:/Users/simeon/Documents/CAT.csv")
PG = pd.read_json("C:/Users/simeon/Documents/PG.txt")
PG_P = pd.read_csv("C:/Users/simeon/Documents/PG.csv")
PFE= pd.read_json("C:/Users/simeon/Documents/PFE.txt")
PFE_P = pd.read_csv("C:/Users/simeon/Documents/PFE.csv")
MSFT = pd.read_json("C:/Users/simeon/Documents/MSFT.txt")
MSFT_P = pd.read_csv("C:/Users/simeon/Documents/MSFT.csv")
MRK = pd.read_json("C:/Users/simeon/Documents/MRK.txt")
MRK_P = pd.read_csv("C:/Users/simeon/Documents/MRK.csv")
MMM= pd.read_json("C:/Users/simeon/Documents/MMM.txt")
MMM_P = pd.read_csv("C:/Users/simeon/Documents/MMM.csv")
JPM = pd.read_json("C:/Users/simeon/Documents/JPM.txt")
JPM_P = pd.read_csv("C:/Users/simeon/Documents/JPM.csv")
JNJ = pd.read_json("C:/Users/simeon/Documents/JNJ.txt")
JNJ_P = pd.read_csv("C:/Users/simeon/Documents/JNJ.csv")
INTC = pd.read_json("C:/Users/simeon/Documents/INTC.txt")
INTC_P = pd.read_csv("C:/Users/simeon/Documents/INTC.csv")
IBM = pd.read_json("C:/Users/simeon/Documents/IBM.txt")
IBM_P = pd.read_csv("C:/Users/simeon/Documents/IBM.csv")
HD = pd.read_json("C:/Users/simeon/Documents/HD.txt")
HD_P = pd.read_csv("C:/Users/simeon/Documents/HD.csv")
GE = pd.read_json("C:/Users/simeon/Documents/GE.txt")
GE_P = pd.read_csv("C:/Users/simeon/Documents/GE.csv")
XOM = pd.read_json("C:/Users/simeon/Documents/XOM.txt")
XOM_P = pd.read_csv("C:/Users/simeon/Documents/XOM.csv")
DD = pd.read_json("C:/Users/simeon/Documents/DD.txt")
DD_P = pd.read_csv("C:/Users/simeon/Documents/DD.csv")
KO = pd.read_json("C:/Users/simeon/Documents/KO.txt")
KO_P = pd.read_csv("C:/Users/simeon/Documents/KO.csv")
CVX = pd.read_json("C:/Users/simeon/Documents/CVX.txt")
CVX_P = pd.read_csv("C:/Users/simeon/Documents/CVX.csv")
BA = pd.read_json("C:/Users/simeon/Documents/BA.txt")
BA_P = pd.read_csv("C:/Users/simeon/Documents/BA.csv")
AXP = pd.read_json("C:/Users/simeon/Documents/AXP.txt")
AXP_P = pd.read_csv("C:/Users/simeon/Documents/AXP.csv")
FB = pd.read_json("C:/Users/simeon/Documents/FB.txt")
FB_P = pd.read_csv("C:/Users/simeon/Documents/FB.csv")
QQQ = pd.read_json("C:/Users/simeon/Documents/QQQ.txt")
QQQ_P = pd.read_csv("C:/Users/simeon/Documents/QQQ.csv")
SPY = pd.read_json("C:/Users/simeon/Documents/SPY.txt")
SPY_P = pd.read_csv("C:/Users/simeon/Documents/SPY.csv")




































    
shebang(NKE, NKE_P, "$NKE")  
NKE = F[:]     
shebang(MCD, MCD_P, "$MCD")  
MCD = F[:]  
shebang(DIS, DIS_P, "$DIS")  
DIS = F[:]  
shebang(WMT, WMT_P, "WMT")  
WMT = F[:]   
shebang(VZ, VZ_P, "$VZ")  
VZ = F[:] 
shebang(V, V_P, "$V")  
V = F[:] 
shebang(UTX, UTX_P, "$UTX")  
UTX = F[:] 
shebang(UNH, UNH_P, "$UNH")  
UNH = F[:] 
shebang(TRV, TRV_P, "$TRV")  
TRV = F[:] 
shebang(CSCO, CSCO_P, "$CSCO")  
CSCO = F[:] 
shebang(CAT, CAT_P, "$CAT")  
CAT = F[:]
shebang(PG, PG_P, "$PG")  
PG = F[:]
shebang(PFE, PFE_P, "$PFE")  
PFE = F[:]
shebang(MSFT, MSFT_P, "$MSFT")  
MSFT = F[:]
shebang(MRK, MRK_P, "$MRK")  
MRK = F[:]
shebang(MMM, MMM_P, "$MMM")  
MMM = F[:]
shebang(JPM, JPM_P, "$JPM")  
JPM = F[:]
shebang(JNJ, JNJ_P, "$JNJ")  
JNJ = F[:]
shebang(INTC, INTC_P, "$INTC")  
INTC = F[:]
shebang(IBM, IBM_P, "$IBM")  
IBM = F[:]
shebang(HD, HD_P, "$HD")  
HD = F[:]
shebang(GS, GS_P, "$GS")  
GS = F[:]
shebang(GE, GE_P, "$GE")  
GE = F[:]
shebang(XOM, XOM_P, "$XOM")  
XOM = F[:]
shebang(DD, DD_P, "$DD")  
DD = F[:]
shebang(KO, KO_P, "$KO")  
KO = F[:]
shebang(CVX, CVX_P, "$CVX")  
CVX = F[:]
shebang(BA, BA_P, "$BA")  
BA = F[:]
shebang(AXP, AXP_P, "$AXP")  
AXP = F[:]
shebang(FB, FB_P, "$FB")  
FB = F[:]
shebang(QQQ, QQQ_P, "$QQQ")  
QQQ = F[:]
shebang(SPY, SPY_P, "$SPY")  
SPY = F[:]
































































































#GS0.to_csv('C:/Users/simeon/Documents/GS0_test.csv')



def W_M(data):
    df_new = data["tweets_no_stop"][data["tweets_no_stop"].notnull()]
    all_words = [] 
    for i in range(len(df_new)):
        print(i)
        a = df_new[i].rsplit()
        all_words.extend(a) 
    global add
    add = all_words
        
    
full_list = []   










W_M(NKE) 
full_list.extend(add)  

    
W_M(MCD) 
full_list.extend(add)    
    
W_M(DIS) 
full_list.extend(add) 
W_M(WMT) 
full_list.extend(add)
W_M(VZ) 
full_list.extend(add)
W_M(V) 
full_list.extend(add)
W_M(UTX) 
full_list.extend(add)
W_M(UNH) 
full_list.extend(add)
W_M(TRV) 
full_list.extend(add)
W_M(CSCO) 
full_list.extend(add)
W_M(CAT) 
full_list.extend(add)
W_M(PG) 
full_list.extend(add)
W_M(PFE) 
full_list.extend(add)
W_M(MSFT) 
full_list.extend(add)
W_M(MRK) 
full_list.extend(add)
W_M(MMM) 
full_list.extend(add)
W_M(JPM) 
full_list.extend(add)
W_M(JNJ) 
full_list.extend(add)
W_M(INTC) 
full_list.extend(add)
W_M(IBM) 
full_list.extend(add)
W_M(HD) 
full_list.extend(add)
W_M(GS) 
full_list.extend(add)
W_M(GE) 
full_list.extend(add)
W_M(XOM) 
full_list.extend(add)
W_M(DD) 
full_list.extend(add)
W_M(KO) 
full_list.extend(add)
W_M(CVX) 
full_list.extend(add)
W_M(BA) 
full_list.extend(add)
W_M(AXP) 
full_list.extend(add)
W_M(FB) 
full_list.extend(add)
W_M(QQQ) 
full_list.extend(add)
W_M(SPY) 
full_list.extend(add)



 
th = Counter(full_list) 












for key, count in dropwhile(lambda key_count: key_count[1] >= 25, th.most_common()):
    del th[key]
    
    
everything = list(set(full_list))   
    
keep = list(th.keys())





l2s = set(keep)

rid = [x for x in everything if x not in l2s]

mylist = list(set(rid))




def uncommon_word_rem(Input_data):
    
    Input_data = pd.DataFrame(Input_data)
    
    
    
    
    pat = r'\b(?:{})\b'.format('|'.join(mylist))
    Input_data['bodyn'] = Input_data["tweets_no_stop"].astype(str).str.replace(pat, '')
    Input_data["tweets_no_stop"] = Input_data['bodyn'].str.replace(r'\s+', ' ')
    global new
    new = Input_data[:]



uncommon_word_rem(NKE)
NKE = new
uncommon_word_rem(MCD)
MCD = new
uncommon_word_rem(DIS)
DIS = new
uncommon_word_rem(WMT)
WMT = new
uncommon_word_rem(VZ)
VZ = new
uncommon_word_rem(V)
V = new
uncommon_word_rem(UTX)
UTX = new
uncommon_word_rem(UNH)
UNH = new
uncommon_word_rem(TRV)
TRV = new
uncommon_word_rem(CSCO)
CSCO = new
uncommon_word_rem(CAT)
CAT = new
uncommon_word_rem(PG)
PG = new
uncommon_word_rem(PFE)
PFE = new
uncommon_word_rem(MSFT)
MSFT = new
uncommon_word_rem(MRK)
MRK = new
uncommon_word_rem(MMM)
MMM = new
uncommon_word_rem(JPM)
JPM = new
uncommon_word_rem(JNJ)
JNJ = new
uncommon_word_rem(INTC)
INTC = new
uncommon_word_rem(IBM)
IBM = new
uncommon_word_rem(HD)
HD = new

uncommon_word_rem(GE)
GE = new
uncommon_word_rem(XOM)
XOM = new
uncommon_word_rem(DD)
DD = new
uncommon_word_rem(KO)
KO = new
uncommon_word_rem(CVX)
CVX = new
uncommon_word_rem(BA)
BA= new
uncommon_word_rem(AXP)
AXP = new
uncommon_word_rem(FB)
FB = new
uncommon_word_rem(QQQ)
QQQ = new
uncommon_word_rem(SPY)
SPY = new



uncommon_word_rem(GS)
GS = new[:]

tfidf = TfidfVectorizer(max_features=10000, lowercase=True, analyzer='word',
stop_words= 'english',ngram_range=(1,1))



train_vect = tfidf.fit_transform(GS["tweets_no_stop"]
)
dense = train_vect.todense()
dense = pd.DataFrame(dense)
GS = pd.concat([GS, dense.set_index(GS.index)], axis=1)


GS['1buy'] = np.where(GS['1_day_return']>=0, 1, 0)
GS['3buy'] = np.where(GS['3_day_return']>=0, 1, 0)
GS['5buy'] = np.where(GS['5_day_return']>=0, 1, 0)





x =GS[np.isfinite(GS['tot_tweet'])]

y_1=pd.DataFrame(x['1buy'])
y_3=pd.DataFrame(x['3buy'])
y_5=pd.DataFrame(x['5buy'])

x= x.drop(["Adj Close","mv10","1buy","3buy","5buy","Open","High","Low","Volume","polarity",'1_day_return',"CLOSE","3_day_return","5_day_return","tweets","roll_avg","percent_change","tweets_no_stop","bodyn"], axis=1)          

#GS.to_csv(r'C:/Users/simeon/Documents/GS_test.csv')
x.dtypes

import pandas as pd
from string import ascii_letters
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math 
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegressionCV
from numpy import *
from numpy import isnan

#y_1.to_csv(r'C:/Users/simeon/Documents/y_1_test.csv')
#X_1.to_csv(r'C:/Users/simeon/Documents/x_1_test.csv')

where_are_NaNs = isnan(x)
x[where_are_NaNs] = 0

#1,000 features for 1 days returns
X_1=SelectKBest(chi2,k=1000).fit_transform(x,y_1)
X_1=pd.DataFrame(X_1)

#1,000 features for 3 days returns
X_3=SelectKBest(chi2,k=1000).fit_transform(x,y_3)
X_3=pd.DataFrame(X_3)


#1,000 features for 5 days returns

X_5=SelectKBest(chi2,k=1000).fit_transform(x,y_5)
X_5=pd.DataFrame(X_5)

# split 
X_train_1, X_test_1, y_train_1, y_test_1 =X_1[:756], X_1[756:], y_1[:756], y_1[756:]

X_train_3, X_test_3, y_train_3, y_test_3 =X_3[:588], X_3[589:], y_3[:588], y_3[589:]
X_train_5, X_test_5, y_train_5, y_test_5 =X_5[:588], X_5[589:], y_5[:588], y_5[589:]




### 1 day ahead 

X_train=X_train_1
y_train=y_train_1

X_test=X_test_1
y_test=y_test_1







clf = GaussianNB()
clf.fit(X_train, np.ravel(y_train))
y_pred = clf.predict(X_test)
cm=confusion_matrix(y_test, y_pred)
print( "Accurancy=", clf.score(X_test, y_test) *100, "%")


